In [ ]:
import matplotlib.pyplot as plt
from glob import glob
import cv2
import random
import os
%matplotlib inline

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model

from IPython.display import SVG, Image
import tensorflow as tf
print("Tensorflow version:", tf.__version__)


In [ ]:
images = glob("/kaggle/input/fer2013/train/**/*.**")

for i in range(9):
    image = random.choice(images)
    plt.figure(figsize=(12, 12))
    plt.subplot(331 + i)
    plt.imshow(cv2.imread(image))
    plt.axis('off')


In [ ]:
img_size = 48
batch_size = 64

datagen_train = ImageDataGenerator(
    rescale=1./255,  # Rescale to [0, 1]
    rotation_range=10,  # Random rotation
    width_shift_range=0.1,  # Random horizontal shift
    height_shift_range=0.1,  # Random vertical shift
    shear_range=0.1,  # Shear transformation
    zoom_range=0.1,  # Random zoom
    horizontal_flip=True,  # Random horizontal flip
    fill_mode='nearest'  # Fill pixels when transforming
)


datagen_validation = ImageDataGenerator()
train_validation = datagen_train.flow_from_directory(
    "/kaggle/input/fer2013/test/",
    target_size=(img_size, img_size),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True
)


MODELING

In [ ]:
def Convolution(input_tensor, filters, kernel_size):
    x = Conv2D(filters=filters, kernel_size=kernel_size, padding="same")(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Dropout(0.25)(x)
    return x

In [ ]:
def Dense_f(input_tensor, nodes):
    x = Dense(nodes)(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.25)(x)
    return x


In [ ]:
def model_fer(input_shape):
    inputs = Input(input_shape)
    conv_1 = Convolution(inputs, 32, (3, 3))
    conv_2 = Convolution(inputs, 64, (5, 5))
    conv_3 = Convolution(inputs, 128, (3, 3))
    
    flatten = Flatten()(conv_3)
    dense_1 = Dense_f(flatten, 256)


In [ ]:
def model_fer(input_shape):
    inputs = Input(input_shape)
    conv_1 = Convolution(inputs, 32, (3, 3))
    conv_2 = Convolution(inputs, 64, (5, 5))
    
    flatten = Flatten()(conv_2)
    dense_1 = Dense_f(flatten, 256)
    
    output = Dense(7, activation="softmax")(dense_1)
    model = Model(inputs=[inputs], outputs=[output])
    
    model.compile(loss=['categorical_crossentropy'], optimizer='adam', metrics=['accuracy'])
    
    return model

In [ ]:
# Model creation and summary
model = model_fer((48, 48, 1))
print(model)  # Should print the model structure or details, not None
if model is not None:
    model.summary()
else:
    print("Model is None, check the model_fer function.")


In [ ]:
# Train and validation directories (replace with actual paths)
train_dir = "/kaggle/input/fer2013/train/"  # Path to training data
validation_dir = "/kaggle/input/fer2013/test/"  # Path to validation data

# Train generator
train_generator = datagen_train.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True
)

# Validation generator
validation_generator = datagen_validation.flow_from_directory(
    validation_dir,
    target_size=(img_size, img_size),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

# Get the number of classes from the training generator
num_classes = train_generator.num_classes
print("Number of classes:", num_classes)



In [ ]:
# Training parameters
epochs = 15
steps_per_epoch = train_validation.n // train_validation.batch_size
validation_steps = validation_generator.n // validation_generator.batch_size

# Model checkpoint callback to save the best model
checkpoint = ModelCheckpoint("model_weights.weights.h5", monitor="val_accuracy", save_weights_only=True, mode="max", verbose=1)
callbacks = [checkpoint]

# Training the model
history = model.fit(
    x=train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=callbacks
)


MODEL EVALUATION

In [ ]:
model.evaluate(validation_generator)

In [ ]:
PLOTTING LINE

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.subplots_adjust(top=1.0, bottom=0.0, right=0.95, left=0.0, hspace=0.25, wspace=0.35)

In [ ]:
img_size = 48
batch_size = 32

# Data generators
datagen_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen_validation = ImageDataGenerator(rescale=1./255)

# Convolution block function
def Convolution(input_tensor, filters, kernel_size):
    x = Conv2D(filters=filters, kernel_size=kernel_size, padding="same")(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    return x

# Dense block function
def Dense_f(input_tensor, nodes):
    x = Dense(nodes)(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.25)(x)
    return x

# Model definition
def model_fer(input_shape):
    inputs = Input(input_shape)
    conv_1 = Convolution(inputs, 32, (3, 3))
    conv_2 = Convolution(conv_1, 64, (5, 5))
    conv_3 = Convolution(conv_2, 128, (3, 3))
    
    flatten = Flatten()(conv_3)
    dense_1 = Dense_f(flatten, 256)
    output = Dense(7, activation="softmax")(dense_1)
    
    model = Model(inputs=[inputs], outputs=[output])
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

# Model creation and summary
model = model_fer((48, 48, 1))
model.summary()

# Train and validation directories (replace with actual paths)
train_dir = "/kaggle/input/fer2013/train/"
validation_dir = "/kaggle/input/fer2013/test/"

# Train generator
train_generator = datagen_train.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True
)

# Validation generator
validation_generator = datagen_validation.flow_from_directory(
    validation_dir,
    target_size=(img_size, img_size),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

# Number of classes
num_classes = train_generator.num_classes
print("Number of classes:", num_classes)

# Training parameters
epochs = 15
steps_per_epoch = train_generator.n // train_generator.batch_size
validation_steps = validation_generator.n // validation_generator.batch_size

# Model checkpoint callback
checkpoint = ModelCheckpoint("model_weights.weights.h5", monitor="val_accuracy", save_weights_only=True, mode="max", verbose=1)
callbacks = [checkpoint]

# Training the model
history = model.fit(
    x=train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=callbacks
)

# Model evaluation
model.evaluate(validation_generator)

# Plotting the losses
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.subplots_adjust(top=1.0, bottom=0.0, right=0.95, left=0.0, hspace=0.25, wspace=0.35)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.subplots_adjust(top=1.0, bottom=0.0, right=0.95, left=0.0, hspace=0.25, wspace=0.35)

In [ ]:
model_json = model.to_json()
with open("model_a.json", "w") as json_file:
    json_file.write(model_json)
